In [11]:
import heapq

def identity(numRows, numCols, val=1, rowStart=0):
   return [[(val if i == j else 0) for j in range(numCols)]
               for i in range(rowStart, numRows)]

def standardForm(cost, greaterThans=[], gtThreshold=[], lessThans=[], ltThreshold=[],
                equalities=[], eqThreshold=[], maximization=True):
   newVars = 0
   numRows = 0
   if gtThreshold != []:
      newVars += len(gtThreshold)
      numRows += len(gtThreshold)
   if ltThreshold != []:
      newVars += len(ltThreshold)
      numRows += len(ltThreshold)
   if eqThreshold != []:
      numRows += len(eqThreshold)

   if not maximization:
      cost = [-x for x in cost]

   if newVars == 0:
      return cost, equalities, eqThreshold

   newCost = list(cost) + [0] * newVars

   constraints = []
   threshold = []

   oldConstraints = [(greaterThans, gtThreshold, -1), (lessThans, ltThreshold, 1),
                     (equalities, eqThreshold, 0)]

   offset = 0
   for constraintList, oldThreshold, coefficient in oldConstraints:
      constraints += [c + r for c, r in zip(constraintList,
         identity(numRows, newVars, coefficient, offset))]

      threshold += oldThreshold
      offset += len(oldThreshold)

   return newCost, constraints, threshold


def dot(a,b):
   return sum(x*y for x,y in zip(a,b))

def column(A, j):
   return [row[j] for row in A]

def transpose(A):
   return [column(A, j) for j in range(len(A[0]))]

def isPivotCol(col):
   return (len([c for c in col if c == 0]) == len(col) - 1) and sum(col) == 1

def variableValueForPivotColumn(tableau, column):
   pivotRow = [i for (i, x) in enumerate(column) if x == 1][0]
   return tableau[pivotRow][-1]

def initialTableau(c, A, b):
   tableau = [row[:] + [x] for row, x in zip(A, b)]
   tableau.append([ci for ci in c] + [0])
   return tableau


def primalSolution(tableau):

   columns = transpose(tableau)
   indices = [j for j, col in enumerate(columns[:-1]) if isPivotCol(col)]
   return [(colIndex, variableValueForPivotColumn(tableau, columns[colIndex]))
            for colIndex in indices]


def objectiveValue(tableau):
   return -(tableau[-1][-1])


def canImprove(tableau):
   lastRow = tableau[-1]
   return any(x > 0 for x in lastRow[:-1])



def moreThanOneMin(L):
   if len(L) <= 1:
      return False

   x,y = heapq.nsmallest(2, L, key=lambda x: x[1])
   return x == y


def findPivotIndex(tableau):
   
   column_choices = [(i,x) for (i,x) in enumerate(tableau[-1][:-1]) if x > 0]
   column = min(column_choices, key=lambda a: a[1])[0]

  
   if all(row[column] <= 0 for row in tableau):
      raise Exception('Linear program is unbounded.')

   
   quotients = [(i, r[-1] / r[column])
      for i,r in enumerate(tableau[:-1]) if r[column] > 0]

   if moreThanOneMin(quotients):
      raise Exception('Linear program is degenerate.')

   
   row = min(quotients, key=lambda x: x[1])[0]

   return row, column


def pivotAbout(tableau, pivot):
   i,j = pivot

   pivotDenom = tableau[i][j]
   tableau[i] = [x / pivotDenom for x in tableau[i]]

   for k,row in enumerate(tableau):
      if k != i:
         pivotRowMultiple = [y * tableau[k][j] for y in tableau[i]]
         tableau[k] = [x - y for x,y in zip(tableau[k], pivotRowMultiple)]


def simplex(c, A, b):
   tableau = initialTableau(c, A, b)
   print("Initial tableau:")
   for row in tableau:
      print(row)
   print()

   while canImprove(tableau):
      pivot = findPivotIndex(tableau)
      print("Next pivot index is=%d,%d \n" % pivot)
      pivotAbout(tableau, pivot)
      print("Tableau after pivot:")
      for row in tableau:
         print(row)
      print()

   return tableau, primalSolution(tableau), objectiveValue(tableau)


if __name__ == "__main__":
   c = [7, 6, 1]
   A = [[0.049, 0, 1], [0.705, 1, 0], [0.344, 0, 0]]
   b = [4131, 167.7, 250, 0]

  
   A[0] += [1,0,0,0]
   A[1] += [0,1,0,0]
   A[2] += [0,0,1,0]
   
   c += [0,0,0,0]

   t, s, v = simplex(c, A, b)
   print(s)
   print(v-2000)

Initial tableau:
[0.049, 0, 1, 1, 0, 0, 0, 4131]
[0.705, 1, 0, 0, 1, 0, 0, 167.7]
[0.344, 0, 0, 0, 0, 1, 0, 250]
[7, 6, 1, 0, 0, 0, 0, 0]

Next pivot index is=0,2 

Tableau after pivot:
[0.049, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 4131.0]
[0.705, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 167.7]
[0.344, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 250.0]
[6.951, 6.0, 0.0, -1.0, 0.0, 0.0, 0.0, -4131.0]

Next pivot index is=1,1 

Tableau after pivot:
[0.049, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 4131.0]
[0.705, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 167.7]
[0.344, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 250.0]
[2.721, 0.0, 0.0, -1.0, -6.0, 0.0, 0.0, -5137.2]

Next pivot index is=1,0 

Tableau after pivot:
[0.0, -0.06950354609929078, 1.0, 1.0, -0.06950354609929078, 0.0, 0.0, 4119.344255319149]
[1.0, 1.4184397163120568, 0.0, 0.0, 1.4184397163120568, 0.0, 0.0, 237.87234042553192]
[0.0, -0.4879432624113475, 0.0, 0.0, -0.4879432624113475, 1.0, 0.0, 168.17191489361704]
[0.0, -3.8595744680851065, 0.0, -1.0, -9.859574468085107, 0.0, 0.0, -5784.450638297872